In [3]:
import sqlite3
conn = sqlite3.connect('good-ol-tickdata.db')
import time

In [4]:
c = conn.cursor()
# No ORMs, just manual table create

c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
conn.commit()

In [5]:
symbol = 'RHAT'
c.execute("SELECT * FROM stocks WHERE symbol = '%s'" % symbol)

# Do this instead
t = ('RHAT',)
c.execute('SELECT * FROM stocks WHERE symbol=?', t)
print(c.fetchone())

# Prepare tick data 
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ] * int(1e5)
print(len(purchases))

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
300000


In [ ]:
start = time.time()
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)
end = time.time()
print('All insert took {} seconds'.format(end-start))

In [ ]:
%%timeit list(c.execute('SELECT * FROM stocks ORDER BY price'))